## 0) 前提整理

* エンジン: MySQL 8.0.40
* 並び順任意なので `ORDER BY` は付けない
* `NOT IN` は使用せず、ウィンドウ関数 / 集約 + 結合で解決
* 判定軸:

  * 「その product_id にとって最初に販売された year」
  * その年に属する全ての販売行（= 行レベルでそのまま残す）

---

## 1) 問題の要点再整理

**テーブル: `Sales`**

```text
sale_id     int      -- (sale_id, year) が PK
product_id  int
year        int
quantity    int
price       int      -- 単価
```

**要求仕様**

* 各 `product_id` ごとに「最初に登場した year（最小 year）」を求める
* その `product_id` について、その最小 year に属する全行を返す

  * つまり「最初の年」の「全売上レコード」を返す（集約しない）
* 出力列:

  * `product_id`
  * `first_year` … 対象行の `year`
  * `quantity`
  * `price`
* 並び順は任意

---

## 2) 最適解（単一クエリ / ウィンドウ関数）

`product_id` 毎に `year` 昇順でランクを付け、最初の年（ランク 1）の行だけを残す、という王道パターンです。

```sql
WITH win AS (
  SELECT
    product_id,
    year,
    quantity,
    price,
    DENSE_RANK() OVER (
      PARTITION BY product_id
      ORDER BY year
    ) AS rnk
  FROM Sales
)
SELECT
  product_id,
  year AS first_year,
  quantity,
  price
FROM win
WHERE rnk = 1;

Runtime 736 ms
Beats 36.82%

```

### ポイント

* `PARTITION BY product_id ORDER BY year`
  → 各商品ごとに販売年で並べ、`DENSE_RANK()` で
  最も早い年の行に `rnk = 1` を振る
* 「同じ product_id・同じ year で複数行」ある場合も、その年の行はすべて `rnk = 1` となる
  → 仕様どおり「最初の年の全 sales 行」が残る
* `ORDER BY` 句なし → 実行計画の自由度を確保しつつ、要件も満たす

---

## 3) 代替解（集約 + 結合 / ウィンドウ未対応環境も意識）

ウィンドウ関数を避ける/重いと判断されるケースでは、**最小年のみを先に集約**し、それに結合するパターンがシンプルで高速です。

```sql
-- 各 product_id の最初の年を集約
WITH first_years AS (
  SELECT
    product_id,
    MIN(year) AS first_year
  FROM Sales
  GROUP BY product_id
)
SELECT
  s.product_id,
  f.first_year,
  s.quantity,
  s.price
FROM Sales AS s
JOIN first_years AS f
  ON  s.product_id = f.product_id
  AND s.year       = f.first_year;

Runtime 724 ms
Beats 41.50%
```

### ポイント

* `first_years` で `product_id` ごとの `MIN(year)` を算出

  * ここは 1 行 / product_id に縮約されるのでデータ量が大きいほど効く
* 元の `Sales` と `product_id` + `year` で内部結合

  * これにより「最初の年の sales 行」だけを抽出
* `NOT IN` などを使わず、安全に実装できる

---

## 4) 要点解説

### 方針

1. **最初の年の特定**

   * ウィンドウ版: `DENSE_RANK() OVER(PARTITION BY product_id ORDER BY year)`
   * 集約版: `MIN(year)` を `GROUP BY product_id` で求める
2. **行のフィルタ**

   * ランク = 1 の行のみ残す（ウィンドウ）
   * `product_id` & `year = first_year` に一致する行のみ残す（結合）
3. **必要列のみ投影**

   * `product_id`, `first_year`, `quantity`, `price` の 4 列に限定

### NULL / 重複の扱い

* この問題設定では `year` が NULL になる前提は通常ないが、もしあり得る場合:

  * 最初の年を「数値として最小」にしたいなら、`WHERE year IS NOT NULL` を前処理に入れる
* 同一 `product_id`・同一 `year` で複数行が存在しうる（問題文にも明記）

  * **集約しない**ことが重要
  * 「最初の年の全ての売上」を出したいので、その年の行はすべて結果に含める

### 安定性と実務的観点

* 並び順は問われていないため `ORDER BY` を付けない方が、オプティマイザに自由度があり高速になりやすい
* 実務では:

  * `product_id, year` にインデックスがあると、どちらの解法でもかなり効く
  * 利用頻度が高いならビューやマテビューで隠蔽するのも手

---

## 5) 計算量（概算）

N = `Sales` の行数、P = `product_id` の種類数（パーティション数）

### ウィンドウ関数版

* パーティションごとにソートが入るため、概ね:

  * 時間計算量: **O(N log N)** 近辺
  * 空間計算量: ソートバッファ分を含め **O(N)** 近辺
* `product_id` ごとの行数が均一なら、`O(P * (N/P log(N/P))) ≒ O(N log(N/P))`

### 集約 + 結合版

* `GROUP BY product_id` での集約: **O(N log N)**（インデックス次第で軽減）
* その結果（P 行）との結合:

  * インデックスがあれば概ね **O(N)** 近似
* いずれもオーダー的にはウィンドウ版と同程度だが、

  * 「first_years がかなり小さい」＋「結合キーに良インデックスあり」
    の場合、集約版が有利になるケースもある

---

## 6) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[入力 Sales テーブル]
  B[集約 各商品ごとの最初の年を算出]
  C[元の Sales と集約結果を結合]
  D[最初の年の行だけが残る]
  E[出力 product_id first_year quantity price]
  A --> B
  A --> C
  B --> C
  C --> D
  D --> E
```

---

どちらも MySQL 8.0.40 で素直に動きますが、

* クエリの読みやすさ重視 → **集約 + 結合版**
* ウィンドウ関数で揃えたい / 他の順位ロジックも追加予定 → **ウィンドウ関数版**

という形で使い分けると扱いやすいと思います。

結論から言うと、**どちらのクエリもロジックとしてはほぼ最適クラス**で、LeetCode のスコアを劇的に押し上げるような「アルゴリズム改善」は正直ほとんどありません。
そのうえで、

* 「書き方を少しシンプルにする案」
* 「理論的には速くなり得るパターン」
* 「実務でのチューニングポイント」

という観点で整理します。

---

## 1. 現状 2 クエリの評価

### ① ウィンドウ関数版

```sql
WITH win AS (
  SELECT
    product_id,
    year,
    quantity,
    price,
    DENSE_RANK() OVER (
      PARTITION BY product_id
      ORDER BY year
    ) AS rnk
  FROM Sales
)
SELECT
  product_id,
  year AS first_year,
  quantity,
  price
FROM win
WHERE rnk = 1;

Runtime 660 ms
Beats 78.09%

```

* 計算量: パーティションごとのソートが入るので概ね **O(N log N)**
* 「最初の年の全行」を出すという要件に対しては非常に素直で可読性も高い
* LeetCode の 736ms / 36.82% は、プラットフォームのノイズもあるので「十分許容範囲」です

### ② 集約 + 結合版

```sql
WITH first_years AS (
  SELECT
    product_id,
    MIN(year) AS first_year
  FROM Sales
  GROUP BY product_id
)
SELECT
  s.product_id,
  f.first_year,
  s.quantity,
  s.price
FROM Sales AS s
JOIN first_years AS f
  ON  s.product_id = f.product_id
  AND s.year       = f.first_year;

Runtime 636 ms
Beats 91.96%

```

* 計算量: `GROUP BY` が **O(N log N)**、結合が **O(N)** 近似 → 全体としても **O(N log N)** クラス
* ロジックは非常にオーソドックスで、実務でもこの書き方をよく使います
* Runtime 724ms / 41.50% なので、ウィンドウ版よりややマシ、という感じですが誤差レベルです

→ どちらも「ちゃんとした解法」で、スコアも全く悪くありません。

---

## 2. もう少しシンプルに書く「第3案」（相関サブクエリ）

LeetCode や面接的には、**相関サブクエリで `MIN(year)` をそのまま使うパターン**もよく出てきます。
MySQL のオプティマイザ次第ですが、場合によってはこれが一番速く出るケースもあります。

```sql
SELECT
  s.product_id,
  s.year AS first_year,
  s.quantity,
  s.price
FROM Sales AS s
WHERE s.year = (
  SELECT MIN(year)
  FROM Sales
  WHERE product_id = s.product_id
);

Time Limit Exceeded
5 / 10 testcases passed
```

### 特徴

* ロジックは非常に明快

  * 「自分の product_id の中で最小 year と同じ年の行だけ残す」
* `NOT IN` は使っていないので、NULL 罠もなし
* インデックスが `product_id, year` に張られている場合は、
  内側の `MIN(year)` がかなり効きやすく、実行計画によっては他案より有利になることもあります

LeetCode 環境ではインデックスをいじれないため「必ず速くなる」とまでは言えませんが、
**SQL の書き方としてはコンパクトで、十分「最適解候補」**です。

---

## 3. ウィンドウ版をあえてチューニングするなら

ウィンドウにかける行数を減らす構成も一応あります。

### 3.1 product_id × year で一度ユニーク化してからウィンドウ

もし「同じ product_id・同じ year の行が大量にある」ようなデータなら、
ウィンドウをかける手前でユニーク化してから join し直す手もあります。

```sql
WITH year_level AS (
  SELECT DISTINCT
    product_id,
    year
  FROM Sales
),
win AS (
  SELECT
    product_id,
    year,
    DENSE_RANK() OVER (
      PARTITION BY product_id
      ORDER BY year
    ) AS rnk
  FROM year_level
)
SELECT
  s.product_id,
  w.year AS first_year,
  s.quantity,
  s.price
FROM Sales AS s
JOIN win AS w
  ON s.product_id = w.product_id
 AND s.year       = w.year
WHERE w.rnk = 1;

Runtime 760 ms
Beats 29.46%

```

* **年レベルでの distinct 行数 << Sales 全行数** なら、
  ウィンドウをかける対象が減るので、理論上は多少有利になります
* ただし、LeetCode のテストデータ規模では

  * `DISTINCT` のコスト
  * 追加の `JOIN` コスト
    が上回る可能性もあり、必ずしも速くなるとは限りません

---

## 4. 実務でのチューニング観点（LeetCode では変更不可ですが）

LeetCode ではインデックス定義は触れませんが、**現場ならまずインデックス設計を見直す**箇所です。

1. 典型的には以下のようなインデックスを検討します

   ```sql
   CREATE INDEX idx_sales_product_year
     ON Sales (product_id, year);
   ```

   これにより

   * `GROUP BY product_id` + `MIN(year)`
   * `WHERE product_id = ... ORDER BY year LIMIT 1`
   * 相関サブクエリの `MIN(year)`

   がすべて「product_id ごとの year の先頭」だけを見にいけるようになります。

2. ウィンドウ関数版でも、`PARTITION BY product_id ORDER BY year` が
   ちょうどそのインデックス順と一致するため、ソートがかなり軽くなります。

---

## 5. 結局「改善の余地」はあるか？

整理すると:

* アルゴリズムクラスとしては

  * 既存 2 クエリも
  * 第3案（相関サブクエリ）も
    いずれも **O(N log N) の妥当な解法**で、これ以上劇的に下げるのは難しい
* LeetCode の Runtime パーセンタイルは

  * サーバ負荷や他ユーザの同時実行状況で ±100ms くらい平気でブレる
  * 36〜41% というのは「十分許容で、改善必須とは言えないレベル」
* その中でやるとすれば

  * 相関サブクエリ版を一度試してみる
  * ウィンドウ版を「distinct year → join 戻し」で組み直してみる
    程度の「微調整」です

なので、**大きな改善余地はほぼないが、書き方を変えた第3案を試す価値はある**、くらいの温度感が現実的だと思います。